# Testing playground for V2 of chatbot with memory.

In [1]:
import sqlite3
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.memory import ConversationBufferMemory

conn = sqlite3.connect('chatbot_database.db')

db = SQLDatabase.from_uri("sqlite:///./chatbot_database.db")
toolkit = SQLDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

memory = ConversationBufferMemory(memory_key="chat_history")

suffix = """Begin!"

Relevant pieces of previous conversation:
{chat_history}
(You do not need to use these pieces of information if not relevant)

Question: {input}
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
{agent_scratchpad}
"""

agent_executor = create_sql_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    suffix=suffix,
    input_variables= ['chat_history', 'input', 'agent_scratchpad'],
    agent_executor_kwargs={'memory':memory}
)

In [2]:
agent_executor.run("What is Alanis Morisette's first album?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:I should query the schema of the Album and Artist tables to see what information I can get.
Action: sql_db_schema
Action Input: Album, Artist
Observation: 
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Artist" (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from Artist table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/
Thought:I should query the Album table to get the title of Alanis Morisette's first album.
Action: sql_d

'Jagged Little Pill'

In [3]:
agent_executor.run("What are the tracks on that album?")



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: ""
Observation: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track
Thought:I should query the schema of the Album and Track tables to see what columns I can use to answer the question.
Action: sql_db_schema
Action Input: "Album, Track"
Observation: 
CREATE TABLE "Album" (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES "Artist" ("ArtistId")
)

/*
3 rows from Album table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(1

'The tracks on the album Jagged Little Pill are All I Really Want, You Oughta Know, Perfect, Hand In My Pocket, Right Through You, Forgiven, You Learn, Head Over Feet, Mary Jane, Ironic, Not The Doctor, Wake Up, and You Oughta Know (Alternate).'

In [4]:
memory.load_memory_variables({})

{'chat_history': "Human: What is Alanis Morisette's first album?\nAI: Jagged Little Pill\nHuman: What are the tracks on that album?\nAI: The tracks on the album Jagged Little Pill are All I Really Want, You Oughta Know, Perfect, Hand In My Pocket, Right Through You, Forgiven, You Learn, Head Over Feet, Mary Jane, Ironic, Not The Doctor, Wake Up, and You Oughta Know (Alternate)."}